## Mapping the pet data

I wanted to try mapping some of the pet data as well. I settled on a chloropleth map using Folium

In [1]:
import folium
from folium import plugins

import json

import pandas as pd

## Chloropleth map of pet density (Seattle only)

We'll start with a chloropleth map showing which zip codes have the highest concentration of pets.

In [2]:
# make sure to export the zip as the right type or this won't work
pets = pd.read_csv('pets_pivot.csv', dtype={'cleaned_zip': object})

# create initial map
seattle_pets_map = folium.Map(location=[47.608013, -122.335167], zoom_start=10, tiles='cartodbpositron')

folium.Choropleth(
    geo_data='geojson_files/seattle-zip-codes.geojson',
    name='chloropleth',
    data=pets,
    columns=['cleaned_zip', 'total_animals'],
    # looks for the info in the geojson file
    key_on='feature.properties.ZCTA5CE10',
    fill_color='YlGnBu', 
    fill_opacity=1, 
    line_opacity=1,
    legend_name='Total number of pets per zip',
    smooth_factor=0).add_to(seattle_pets_map)

# show the map
seattle_pets_map

## Chloropleth of pet density (all WA zips)

As it turns out, the zip codes reflected in that data expand beyond the limits of Seattle proper. (And in fact, there wre some from as far away as Iowa and Texas, but we removed those in the cleaning.) 

In order to show this, we'll use a diferent GeoJSON file that reflects all the zip codes in Washington state.

### Create a new geojSON file

The Washington state file is huge, so it might be nice to make sure we're not loading data we aren't using. So, using (PyGeoj)![https://pypi.org/project/PyGeoj/
] let's create a new file that only shows data for the zip codes that appear in the csv we're using. 

In [3]:
import pygeoj

source_file = pygeoj.load(filepath="geojson_files/wa_state_zips.min.json")

zip_list = pets.cleaned_zip.to_list()

tempfile = pygeoj.new()

for feature in source_file:
    item = feature.properties['ZCTA5CE10']
    if item in zip_list:
        tempfile.add_feature(properties=feature.properties,
                    geometry=feature.geometry )
tempfile.save("geojson_files/select_wa_zip_codes.geojson")

### Create a new map

We'll zoom out a bit more on this one to make sure we can see all the areas represented.

In [4]:
wa_pets_map = folium.Map(location=[47.608013, -122.335167], zoom_start=7, tiles='cartodbpositron')


folium.Choropleth(
    geo_data= 'geojson_files/select_wa_zip_codes.geojson',
    name='chloropleth',
    data=pets,
    columns=['cleaned_zip', 'total_animals'],
    # looks for the info in the geojson file
    key_on='feature.properties.ZCTA5CE10',
    fill_color='YlGnBu', 
    fill_opacity=1, 
    line_opacity=1,
    legend_name='Total number of pets per zip',
    smooth_factor=0).add_to(wa_pets_map)

# display map
wa_pets_map

## Save the maps as assets

This notebook will display the maps, but should we need to use them elsewhere, we'll need the html files.

### HTML Files
This part is easy, as Folium already has a method.

In [5]:
seattle_pets_map.save('seattle_pets_map.html')
wa_pets_map.save('wa_pets_map.html')

### sources

I always find it helpful to collect the resources that helped me along the way!

-https://towardsdatascience.com/choropleth-maps-with-folium-1a5b8bcdd392
-http://comet.lehman.cuny.edu/owen/teaching/datasci/choroplethLab.html
-https://medium.com/analytics-vidhya/create-and-visualize-choropleth-map-with-folium-269d3fd12fa0
-https://towardsdatascience.com/making-3-easy-maps-with-python-fb7dfb1036

#### make sure to export the df data as the right data type for Folium
-https://github.com/python-visualization/folium/issues/1367
-https://stackoverflow.com/questions/62805928/folium-choropleth-not-colorizing-zip-code-areas-at-all/62829226?noredirect=1#comment111107273_62829226
